In [1]:
import math
from simpleautodiff import Dual,Variable,sin

In [2]:
# some examples

def super_complicated_function(x):
    return x * sin(x+6.)**2. / 2. - 2. / 2.**x

# by running the above in mathematica (wolfram alpha)
# direct link: https://www.wolframalpha.com/input/?i=derivative+x+*+sin%28x%2B6.%29**2.+%2F+2.+-+2%2F2**x
def d_super_complicated_function(x):
    return 2**(1 - x) * math.log(2) + 0.5 * sin(x + 6)**2 + x * math.sin(x + 6) * math.cos(x + 6)

for x in [-1., 0.5, 3., 2.]:
    print('Function Input: {}'.format(x))
    print('Function Value: {}'.format(super_complicated_function(x)))
    print('Function Symbolic Derivative: {}'.format(d_super_complicated_function(x)))
    print(super_complicated_function(Dual(x, 1.)))
    print('-'*32)


print('TESTING Variable')

for x in [-1., 0.5, 3., 2.]:
    print('Function Input: {}'.format(x))
    print('Function Value: {}'.format(super_complicated_function(x)))
    print('Function Symbolic Derivative: {}'.format(d_super_complicated_function(x)))
    x_v = Variable(x)
    L = super_complicated_function(x_v)
    print('Variable Function Output Value: {}'.format(L))
    L.backward()
    print('Input value: {}'.format(x_v))
    print('-'*32)


    

Function Input: -1.0
Function Value: -4.459767882269113
Function Symbolic Derivative: 3.504367159953579
< Dual value: -4.459767882269113, derivative: 3.504367159953579 >
--------------------------------
Function Input: 0.5
Function Value: -1.4026444100543694
Function Symbolic Derivative: 1.1084382073126584
< Dual value: -1.4026444100543694, derivative: 1.1084382073126582 >
--------------------------------
Function Input: 3.0
Function Value: 0.004762468816939924
Function Symbolic Derivative: -0.8682732520785479
< Dual value: 0.004762468816939924, derivative: -0.8682732520785477 >
--------------------------------
Function Input: 2.0
Function Value: 0.47882974016169233
Function Symbolic Derivative: 0.5480851436957535
< Dual value: 0.47882974016169233, derivative: 0.5480851436957535 >
--------------------------------
TESTING Variable
Function Input: -1.0
Function Value: -4.459767882269113
Function Symbolic Derivative: 3.504367159953579
Variable Function Output Value: < Variable value: -4.4

In [3]:
def forward_fn(x):
    for n in range(5):
        if n % 2 == 0:
            x = 3.*x
        else:
            x = x**(1./n) + 1./n
        
    return x

x = Variable(2.)
y = forward_fn(x)
print(y)

y.backward()
print(x)

< Variable value: 9.276772529143361, gradient: 0.0 >
< Variable value: 2.0, gradient: 1.1823960755919083 >


In [4]:
def fn(x):
    return x**2 + 0.2*(x-2)**5 + 2*x**3

# initialization
x = Variable(3.)
target = 42.
print('---- Initial Value ----')
print('fn(x): {}'.format(fn(x)))
print('Target: {}'.format(target))
print('intial guess for x: {}'.format(x))


for n in range(20):
    L = (fn(x) - target)**2
    L.backward()
    # gradient descent update
    x.value = x.value - 1e-4 * x.gradient
    # clear the gradients
    x.clear_gradient()
    
print('---- Converged Value ----')    
print('fn(x): {}'.format(fn(x)))
print('Target: {}'.format(target))
print('converged x: {}'.format(x))


'''
# Wolfram alpha minimum: 
# min{(x^2 + 0.2 (x - 2)^5 + 2 x^3 - 42)^2} = 0 at x≈2.60158
'''

---- Initial Value ----
fn(x): < Variable value: 63.2, gradient: 0.0 >
Target: 42.0
intial guess for x: < Variable value: 3.0, gradient: 0.0 >
---- Converged Value ----
fn(x): < Variable value: 42.00014691908245, gradient: 0.0 >
Target: 42.0
converged x: < Variable value: 2.601581019114941, gradient: 0.0 >


'\n# Wolfram alpha minimum: \n# min{(x^2 + 0.2 (x - 2)^5 + 2 x^3 - 42)^2} = 0 at x≈2.60158\n'